### Setup env
This is notebook specific setup, my module path is different on my vm so this is a workaround

In [31]:
import sys
import os

path = os.path.abspath(os.path.join('..'))
if path not in sys.path:
    sys.path.append(path)

### Load Dataset
You can change the type of motion by changing the filepath
Dataset right now just repeats the same motion 100 times

In [32]:
from diffusion.data_loaders.motion_dataset import MotionDataset
dataset = MotionDataset("data/motions/humanoid3d_backflip.txt")
len(dataset), dataset[0], dataset[0].trajectories.shape

Tmp angle [0.0, 0.0, 0.886733, 0.9994119999999997, 0.029214999999999935, 0.017962999999999996, -0.0005250000000000012, 0.0008703311351828238, -0.01099955419316837, 0.0011488037112933414, 0.006692356321057513, 0.28207086609928633, -0.023509972489884845, -0.024408685227982754, -0.14478594217336963, -0.25155797447522277, 0.240463, 0.10908909272941009, -0.20003141435314956, -0.02851573566767156, 0.148934, -0.04895066252132076, 0.07777454918628952, 0.00022592685870272312, -0.014186, 0.013966981294680377, -0.07269661242438823, -0.21047301550502306, -0.04167931361131441, 0.013137096158641143, -0.025583069520627662, -0.027859, -0.023363972687569904, 0.016181526060614963, 0.2111699686528958] 35


(100,
 Batch(trajectories=tensor([[ 0.0000e+00,  0.0000e+00,  8.8673e-01,  9.9941e-01,  2.9215e-02,
           1.7963e-02, -5.2500e-04,  8.7033e-04, -1.1000e-02,  1.1488e-03,
           6.6924e-03,  2.8207e-01, -2.3510e-02, -2.4409e-02, -1.4479e-01,
          -2.5156e-01,  2.4046e-01,  1.0909e-01, -2.0003e-01, -2.8516e-02,
           1.4893e-01, -4.8951e-02,  7.7775e-02,  2.2593e-04, -1.4186e-02,
           1.3967e-02, -7.2697e-02, -2.1047e-01, -4.1679e-02,  1.3137e-02,
          -2.5583e-02, -2.7859e-02, -2.3364e-02,  1.6182e-02,  2.1117e-01],
         [-2.0268e-02, -7.3500e-04,  9.0938e-01,  9.9920e-01,  3.0395e-02,
           2.4696e-02,  8.4250e-03, -4.4505e-05,  2.2081e-03, -3.1660e-03,
           1.1586e-02,  2.9219e-01, -1.8050e-02, -2.5669e-01, -1.8148e-01,
          -4.4781e-01,  2.3335e-01,  3.7712e-01, -2.4100e-01,  1.8741e-01,
           1.7199e-01, -6.5981e-02,  1.4936e-02, -4.7221e-03, -1.4254e-02,
           4.3702e-02,  2.0182e-01, -2.0957e-01, -3.0643e-02, -5.7684e-02,

### Setup Model
Configure your experiment name and savepaths here, they will all be stored under the logs folder later on

In [38]:
import os
import torch
from diffusion.diffuser.utils import Trainer as dTrainer, Config as dConfig

exp_name = "test-backflip"
savepath = f'/home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/{exp_name}'
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

if not os.path.exists(savepath):
    os.makedirs(savepath)
    os.makedirs(os.path.join(savepath, 'sampled_motions'))

In [39]:
from diffusion.diffuser.models.temporal import TemporalUnet 

horizon = dataset[0].trajectories.shape[0]
transition_dim = dataset[0].trajectories.shape[1]

model_config = dConfig(
    TemporalUnet,
    savepath=(savepath, 'model_config.pkl'),
    horizon=horizon,
    transition_dim=transition_dim,
    cond_dim=transition_dim,
    device=device,
)
model = model_config()


[utils/config ] Config: <class 'diffusion.diffuser.models.temporal.TemporalUnet'>
    cond_dim: 35
    horizon: 24
    transition_dim: 35

[ utils/config ] Saved config to: /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/test-backflip/model_config.pkl

[ models/temporal ] Channel dimensions: [(35, 32), (32, 64), (64, 128), (128, 256)]
[(35, 32), (32, 64), (64, 128), (128, 256)]


In [40]:
from diffusion.diffuser.models.diffusion import GaussianDiffusion

# model params, I am only using the very basic ones, some params are for conditioning
n_timesteps = 1000
loss_type = 'l2'
clip_denoised = False
predict_epsilon = False
action_weight = 5
loss_weights = None
loss_discount = 1

diffusion_config = dConfig(
    GaussianDiffusion,
    savepath=(savepath, 'diffusion_config.pkl'),
    horizon=horizon,
    transition_dim=transition_dim,
    n_timesteps=n_timesteps,
    loss_type=loss_type,
    clip_denoised=clip_denoised,
    predict_epsilon=predict_epsilon,
    # loss weighting
    action_weight=action_weight,
    loss_weights=loss_weights,
    loss_discount=loss_discount,
    device=device,
)

diffusion = diffusion_config(model)


[utils/config ] Config: <class 'diffusion.diffuser.models.diffusion.GaussianDiffusion'>
    action_weight: 5
    clip_denoised: False
    horizon: 24
    loss_discount: 1
    loss_type: l2
    loss_weights: None
    n_timesteps: 1000
    predict_epsilon: False
    transition_dim: 35

[ utils/config ] Saved config to: /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/test-backflip/diffusion_config.pkl



### Setup Training

In [41]:
learning_rate = 2e-4
gradient_accumulate_every = 2
ema_decay = 0.995
sample_freq = 2000
save_freq = 2000
n_train_steps = 1e5
n_saves = 5
save_parallel = False
bucket = None
n_reference = 8
train_batch_size = 32

trainer_config = dConfig(
    dTrainer,
    savepath=(savepath, 'trainer_config.pkl'),
    train_batch_size=train_batch_size,
    train_lr=learning_rate,
    gradient_accumulate_every=gradient_accumulate_every,
    ema_decay=ema_decay,
    sample_freq=sample_freq,
    save_freq=save_freq,
    label_freq=int(n_train_steps // n_saves),
    save_parallel=save_parallel,
    results_folder=savepath,
    bucket=bucket,
    n_reference=n_reference,
)

trainer = trainer_config(diffusion, dataset, renderer=None)


[utils/config ] Config: <class 'diffusion.diffuser.utils.training.Trainer'>
    bucket: None
    ema_decay: 0.995
    gradient_accumulate_every: 2
    label_freq: 20000
    n_reference: 8
    results_folder: /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/test-backflip
    sample_freq: 2000
    save_freq: 2000
    save_parallel: False
    train_batch_size: 32
    train_lr: 0.0002

[ utils/config ] Saved config to: /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/test-backflip/trainer_config.pkl



### Test if model and training loop works

In [11]:
import torch
from diffuser.utils import batchify

tunet = model
tunet.to(device)
test_data = dataset[0]
batch = batchify(test_data)
t = torch.randint(0, 1000, (1,), device=device).long().to(device)
res = tunet(batch.trajectories, cond=batch.conditions, time=t, verbose=True)
res, res.shape

x.shape torch.Size([1, 160, 35])
x.shape torch.Size([1, 35, 160])
x.shape torch.Size([1, 32, 160])
1
xfinal.shape torch.Size([1, 32, 80])
x.shape torch.Size([1, 64, 80])
2
xfinal.shape torch.Size([1, 64, 40])
x.shape torch.Size([1, 128, 40])
3
xfinal.shape torch.Size([1, 128, 20])
x.shape torch.Size([1, 256, 20])
4
xfinal.shape torch.Size([1, 256, 20])
xt1.shape torch.Size([1, 256, 20])
xt2.shape torch.Size([1, 256, 20])
xt3.shape torch.Size([1, 256, 20])
pop.shape torch.Size([1, 256, 20])
pop.shape torch.Size([1, 128, 40])
pop.shape torch.Size([1, 64, 80])


(tensor([[[-0.1539,  0.0793, -0.0929,  ..., -0.2377, -0.0208, -0.1989],
          [ 0.0218,  0.3455, -0.0038,  ...,  0.2120, -0.0132,  0.3626],
          [ 0.1062,  0.3715,  0.1296,  ..., -0.2114, -0.2689, -0.3486],
          ...,
          [-0.1029,  0.5232, -0.2429,  ...,  0.1849,  0.0469,  0.3792],
          [ 0.3475,  0.0495,  0.2069,  ...,  0.1758,  0.0420, -0.5588],
          [ 0.1055,  0.2337, -0.1396,  ...,  0.0973, -0.0021,  0.1788]]],
        device='cuda:0', grad_fn=<PermuteBackward0>),
 torch.Size([1, 160, 35]))

In [12]:
from diffuser.utils import report_parameters, batchify

report_parameters(model)

print('Testing forward...', end=' ', flush=True)
x = dataset[0]
batch = batchify(x)
loss, _ = diffusion.loss(batch.trajectories, {})
loss.backward()
print('✓')

[ utils/arrays ] Total parameters: 3.96 M
         downs.3.0.blocks.1.block.0.weight: 327.68 k | Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
         downs.3.1.blocks.0.block.0.weight: 327.68 k | Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
         downs.3.1.blocks.1.block.0.weight: 327.68 k | Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
         ups.0.0.blocks.0.block.0.weight: 327.68 k | Conv1d(512, 128, kernel_size=(5,), stride=(1,), padding=(2,))
         mid_block1.blocks.0.block.0.weight: 327.68 k | Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
         mid_block1.blocks.1.block.0.weight: 327.68 k | Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
         mid_block2.blocks.0.block.0.weight: 327.68 k | Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
         mid_block2.blocks.1.block.0.weight: 327.68 k | Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
         downs.3.0.b

### Train Model
It took me 80s to run 1 epoch and results were pretty good from just 1 epoch

In [9]:
n_steps_per_epoch = 1000
n_epochs = int(n_train_steps // n_steps_per_epoch)
print(n_epochs)
n_epochs = 1

for i in range(n_epochs):
    print(f'Epoch {i} / {n_epochs} | {savepath}')
    trainer.train(n_train_steps=n_steps_per_epoch)

trainer.save(n_epochs)

100
Epoch 0 / 1 | /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/test-cartwheel
[ utils/training ] Saved model to /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/test-cartwheel/state_0.pt
0:   0.3086 | a0_loss:   0.0269 | t:   0.4321
100:   0.0675 | a0_loss:   0.0111 | t:   8.1653
200:   0.0279 | a0_loss:   0.0061 | t:   8.1041
300:   0.0137 | a0_loss:   0.0037 | t:   7.9897
400:   0.0073 | a0_loss:   0.0024 | t:   8.1874
500:   0.0044 | a0_loss:   0.0016 | t:   8.3597
600:   0.0030 | a0_loss:   0.0011 | t:   8.4126
700:   0.0022 | a0_loss:   0.0008 | t:   8.2396
800:   0.0017 | a0_loss:   0.0006 | t:   8.4983
900:   0.0014 | a0_loss:   0.0004 | t:   8.5166
[ utils/training ] Saved model to /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/test-cartwheel/state_1.pt


### (Alternatively) load a checkpoint

In [42]:
# from diffusion.diffuser.utils import load_diffusion
# diffusion_experiment = load_diffusion(
#     "/home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/diffuser", dataset=dataset, epoch=1)

# renderer = diffusion_experiment.renderer
# model = diffusion_experiment.trainer.ema_model

# Optionally load a checkpoint
trainer.load(1)
model = trainer.ema_model

### Sample from model - In Betweening with Blending
Let's try doing a cartwheel while walking again

But this time, we just leave the start and end as walking, we want to see if it can transition from walking to cartwheel and back to walking smoothly

In [14]:
from diffusion.data_loaders.motion_dataset import MotionDataset

walk_dataset = MotionDataset("data/motions/humanoid3d_walk.txt")
len(walk_dataset), walk_dataset[0], walk_dataset[0].trajectories.shape

Tmp angle [0.0, 0.0, 0.847532, 0.998678, 0.01410399999999995, 0.049422999999999995, -0.0006980000000000042, 0.019374995056800275, 0.008037254877450587, -0.09523902811084285, -0.0, 0.0, -0.0, -0.15553532463598624, 0.23919429256424163, 0.20739656997070288, 0.170571, 0.3529631848273465, -0.2610682953696931, -0.24560532144975333, 0.581348, 0.02035205257945668, -0.5175742452141794, -0.11376339039728192, -0.249116, 0.020556236630260034, -0.019534498786735962, 0.0655269812790598, -0.05606350142619236, 0.15209578259548684, 0.1827420948157945, -0.391532, 0.1931167851688944, -0.2978918547932108, -0.08305715225197069] 35


(100,
 Batch(trajectories=tensor([[ 0.0000,  0.0000,  0.8475,  ...,  0.1931, -0.2979, -0.0831],
         [ 0.0415, -0.0047,  0.8466,  ...,  0.1812, -0.2662, -0.1258],
         [ 0.0817, -0.0105,  0.8483,  ...,  0.1293, -0.1646, -0.1874],
         ...,
         [ 0.9535,  0.0164,  0.8741,  ...,  0.0799, -0.2361, -0.1045],
         [ 0.9810,  0.0152,  0.8736,  ...,  0.0750, -0.2538, -0.1148],
         [ 1.0080,  0.0138,  0.8721,  ...,  0.0762, -0.2713, -0.1198]]), conditions={0: tensor([ 0.0000e+00,  0.0000e+00,  8.4753e-01,  9.9868e-01,  1.4104e-02,
          4.9423e-02, -6.9800e-04,  1.9375e-02,  8.0373e-03, -9.5239e-02,
         -0.0000e+00,  0.0000e+00, -0.0000e+00, -1.5554e-01,  2.3919e-01,
          2.0740e-01,  1.7057e-01,  3.5296e-01, -2.6107e-01, -2.4561e-01,
          5.8135e-01,  2.0352e-02, -5.1757e-01, -1.1376e-01, -2.4912e-01,
          2.0556e-02, -1.9534e-02,  6.5527e-02, -5.6064e-02,  1.5210e-01,
          1.8274e-01, -3.9153e-01,  1.9312e-01, -2.9789e-01, -8.3057e-02])}

Make the starting motion walk -> noise -> walk

In [44]:
walk_traj = walk_dataset[0].trajectories
cartwheel_traj = dataset[0].trajectories
noise = torch.randn_like(cartwheel_traj)

starting_motion = torch.cat([walk_traj, noise, walk_traj], dim=0)

walk_traj = walk_traj.unsqueeze(0)
starting_motion.unsqueeze_(0)
walk_traj.shape, starting_motion.shape, noise.shape

(torch.Size([1, 32, 35]), torch.Size([1, 88, 35]), torch.Size([24, 35]))

In [45]:
no_of_walk_frames = walk_traj.shape[1]
no_of_starting_motion_frames = starting_motion.shape[1]

start_idx = no_of_walk_frames
end_idx = no_of_starting_motion_frames - no_of_walk_frames

def apply_conditioning_blending(x): 
    # Assume x has the shape of [1, num_frame, data_dim]
    x[:, :start_idx, :] = walk_traj
    x[:, end_idx:, :] = walk_traj
    return x

start_idx, end_idx

(32, 56)

In [46]:
from diffuser.utils import batchify
test = dataset[0]
batch = batchify(test)
sample = diffusion.p_sample_loop(batch.trajectories.shape, batch.conditions, starting_motion=starting_motion, conditioning_fn=apply_conditioning_blending)


                                                                                                    
1 / 1000 [                                                            ]   0% | 10.7 Hz
t : 999 | vmax : 0.0 | vmin : 0.0
                                                                                                    
                                                                                                    
2 / 1000 [                                                            ]   0% | 17.5 Hz
t : 998 | vmax : 0.0 | vmin : 0.0
                                                                                                    
                                                                                                    
3 / 1000 [                                                            ]   0% | 22.6 Hz
t : 997 | vmax : 0.0 | vmin : 0.0
                                                                                                    
                              

### Save results to logs

In [47]:
import numpy as np


def save_motions(sample, output_dir, filename="motion.npy"):
    filepath = os.path.join(output_dir, filename)
    pos_data = sample.trajectories.squeeze(0).cpu().numpy()
    np.save(filepath, pos_data)
    print(f"Motion saved as {filename}")


save_motions(sample, f"{savepath}/sampled_motions", filename="walk-backflip-walk-motion.npy")

Motion saved as walk-backflip-walk-motion.npy


: 